In [1]:
from IPython.display import display, Image
from PIL import Image as PIL_Image
from io import BytesIO
import numpy as np
from IPython.display import display, Image

def display_img_array(ima, **kwargs):
    if ima.dtype == np.float32 or ima.dtype == np.float64:
        ima = (np.clip(ima, 0., 1.)*255).astype(np.uint8)
    im = PIL_Image.fromarray(ima)
    bio = BytesIO()
    im.save(bio, format='png')
    display(Image(bio.getvalue(), format='png', **kwargs))

IMG_C, IMG_W, IMG_H=1,28,28
def show(x):
    x = x.reshape(-1, IMG_C, IMG_H, IMG_W)
    w = min(x.shape[0]*100, 800)
    x = x.swapaxes(0, 1).swapaxes(1,2)
    if IMG_C == 1:
        img = x.reshape(IMG_H, -1)
    else:
        x = x.reshape(IMG_C, IMG_H, -1)
        img = np.swapaxes(x, 0, 1)
        img = np.swapaxes(img, 1, 2)
    display_img_array(img, width=w) 

In [2]:
from keras.utils import to_categorical
from keras import layers, models, optimizers ,Model
from keras.layers import Input 
def load_mnist():
    # the data, shuffled and split between train and test sets
    from keras.datasets import mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.
    x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.
    y_train = to_categorical(y_train.astype('float32'))
    y_test = to_categorical(y_test.astype('float32'))
    return (x_train, y_train), (x_test, y_test)
(x_train, y_train), (x_test, y_test) = load_mnist()

Using TensorFlow backend.
/home/yiwen/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
from keras import backend as K
from keras.layers import Activation 
from keras.utils.generic_utils import get_custom_objects
def swish(x):
    return K.sigmoid(x)*x
get_custom_objects().update({'swish': Activation(swish )})

In [4]:
D=models.Sequential()
D.add(layers.Conv2D(input_shape=(28, 28, 1),filters=256,kernel_size=(3,3),activation='relu',padding='same'))
D.add(layers.Dropout(0.3))
D.add(layers.MaxPool2D())
D.add(layers.Conv2D(filters=10,kernel_size=(3,3),activation='relu',padding='same'))
D.add(layers.Dropout(0.3))
D.add(layers.Conv2D(filters=10,kernel_size=(14,14),activation='softmax',padding='valid'))
#D.add(layers.Activation('softmax'))
D.add(layers.Flatten())
D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 256)       2560      
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 10)        23050     
_________________________________________________________________
dropout_2 (Dropout)          (None, 14, 14, 10)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1, 1, 10)          19610     
_________________________________________________________________
flatten_1 (Flatten)          (None, 10)                0         
Total para

In [7]:
D.compile(loss='mse',optimizer='Adam',metrics=['accuracy'])
D.fit(x_train,y_train,epochs=10,batch_size=300)
D.evaluate(x_test,y_test)

Epoch 1/10
60000/60000 [==============================] - 20s 332us/step - loss: 0.0014 - acc: 0.9910
Epoch 2/10
60000/60000 [==============================] - 20s 329us/step - loss: 0.0014 - acc: 0.9914
Epoch 3/10
60000/60000 [==============================] - 20s 329us/step - loss: 0.0015 - acc: 0.9907
Epoch 4/10
60000/60000 [==============================] - 20s 329us/step - loss: 0.0013 - acc: 0.9916
Epoch 5/10
60000/60000 [==============================] - 20s 330us/step - loss: 0.0013 - acc: 0.9918
Epoch 6/10
60000/60000 [==============================] - 20s 330us/step - loss: 0.0012 - acc: 0.9922
Epoch 7/10
60000/60000 [==============================] - 20s 329us/step - loss: 0.0013 - acc: 0.9920
Epoch 8/10
60000/60000 [==============================] - 20s 330us/step - loss: 0.0012 - acc: 0.9925
Epoch 9/10
60000/60000 [==============================] - 20s 329us/step - loss: 0.0012 - acc: 0.99230s - loss: 0.0012 - acc: 
Epoch 10/10
10000/10000 [==============================] 

[0.001602584640310382, 0.98970000000000002]

In [8]:
D_train=D.predict(x_train)

In [9]:
Decoder=models.Sequential()
Decoder.add(layers.Dense(input_shape=(10,),units=1960,activation='swish'))
Decoder.add(layers.Dropout(0.3))
Decoder.add(layers.Dense(units=1960,activation='swish'))
Decoder.add(layers.Dropout(0.3))
Decoder.add(layers.Dense(units=1960,activation='swish'))
Decoder.add(layers.Dropout(0.3))
Decoder.add(layers.Dense(units=512,activation='swish'))
Decoder.add(layers.Dropout(0.3))
Decoder.add(layers.Dense(units=784,activation='relu'))
Decoder.add(layers.Reshape((28,28,1)))
Decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1960)              21560     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1960)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1960)              3843560   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1960)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1960)              3843560   
_________________________________________________________________
dropout_5 (Dropout)          (None, 1960)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               1004032   
__________

In [14]:
Decoder.compile(loss='mae',optimizer='Adam')
Decoder.fit(D_train,x_train,epochs=20,batch_size=1000)

Epoch 1/20
60000/60000 [==============================] - 2s 40us/step - loss: 0.1024
Epoch 2/20
60000/60000 [==============================] - 2s 35us/step - loss: 0.1024
Epoch 3/20
60000/60000 [==============================] - 2s 35us/step - loss: 0.1024
Epoch 4/20
60000/60000 [==============================] - 2s 35us/step - loss: 0.1024
Epoch 5/20
60000/60000 [==============================] - 2s 36us/step - loss: 0.1024
Epoch 6/20
60000/60000 [==============================] - 2s 35us/step - loss: 0.1024
Epoch 7/20
60000/60000 [==============================] - 2s 35us/step - loss: 0.1024
Epoch 8/20
60000/60000 [==============================] - 2s 36us/step - loss: 0.1024
Epoch 9/20
60000/60000 [==============================] - 2s 35us/step - loss: 0.1024
Epoch 10/20
60000/60000 [==============================] - 2s 35us/step - loss: 0.1024
Epoch 11/20
60000/60000 [==============================] - 2s 35us/step - loss: 0.1024
Epoch 12/20
60000/60000 [===========================

In [11]:
print('還原圖')
reco=Decoder.predict(D_train)
show(reco[1:11])
print('Uniform生成')
a = np.random.random((10,10))
reco = Decoder.predict(a)
show(reco)
print('Normal生成')
a = np.random.normal(0,1,(10,10))
reco = Decoder.predict(a)
show(reco)

還原圖


Uniform生成


Normal生成


In [15]:
XD_train=np.column_stack((y_train,D_train))

In [115]:
YDecoder=models.Sequential()
YDecoder.add(layers.Dense(input_shape=(20,),units=512,activation='relu'))
YDecoder.add(layers.Dropout(0.5))
YDecoder.add(layers.Dense(units=1024,activation='swish'))
YDecoder.add(layers.Dropout(0.5))
YDecoder.add(layers.Dense(units=784,activation='sigmoid'))
YDecoder.add(layers.Reshape((28,28,1)))
YDecoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 512)               10752     
_________________________________________________________________
dropout_33 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dropout_34 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_42 (Dense)             (None, 784)               803600    
_________________________________________________________________
reshape_10 (Reshape)         (None, 28, 28, 1)         0         
Total params: 1,339,664
Trainable params: 1,339,664
Non-trainable params: 0
_________________________________________________________________


In [116]:
from keras import optimizers

# All parameter gradients will be clipped to
# a maximum value of 0.5 and
# a minimum value of -0.5.
sgd = optimizers.SGD(lr=100)

YDecoder.compile(loss='mae',optimizer=sgd)
YDecoder.fit(XD_train,x_train,epochs=500,batch_size=6000)

Epoch 1/500
60000/60000 [==============================] - 1s 23us/step - loss: 0.2734
Epoch 2/500
60000/60000 [==============================] - 0s 7us/step - loss: 0.1268
Epoch 3/500
60000/60000 [==============================] - 0s 7us/step - loss: 0.1262A: 0s - loss: 0.1
Epoch 4/500
60000/60000 [==============================] - 0s 7us/step - loss: 0.1252
Epoch 5/500
60000/60000 [==============================] - 0s 7us/step - loss: 0.1235
Epoch 6/500
60000/60000 [==============================] - 0s 7us/step - loss: 0.1198
Epoch 7/500
60000/60000 [==============================] - 0s 7us/step - loss: 0.1167
Epoch 8/500
60000/60000 [==============================] - 0s 7us/step - loss: 0.1135
Epoch 9/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1113
Epoch 10/500
60000/60000 [==============================] - 0s 7us/step - loss: 0.1099
Epoch 11/500
60000/60000 [==============================] - 0s 7us/step - loss: 0.1090
Epoch 12/500
60000/60000 [========

60000/60000 [==============================] - 0s 7us/step - loss: 0.1029
Epoch 96/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1029
Epoch 97/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1029
Epoch 98/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1029
Epoch 99/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1029
Epoch 100/500
60000/60000 [==============================] - 0s 7us/step - loss: 0.1029
Epoch 101/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1029
Epoch 102/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1029
Epoch 103/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1029
Epoch 104/500
60000/60000 [==============================] - 0s 7us/step - loss: 0.1029
Epoch 105/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1029
Epoch 106/500
60000/60000 [=======================

60000/60000 [==============================] - 0s 6us/step - loss: 0.1023
Epoch 282/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1023
Epoch 283/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1023
Epoch 284/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1023
Epoch 285/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1023
Epoch 286/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1023
Epoch 287/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1023
Epoch 288/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1023
Epoch 289/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1023
Epoch 290/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1023
Epoch 291/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1023
Epoch 292/500
60000/60000 [===================

Epoch 467/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1022
Epoch 468/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1022
Epoch 469/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1022
Epoch 470/500
60000/60000 [==============================] - 0s 7us/step - loss: 0.1022
Epoch 471/500
60000/60000 [==============================] - 0s 7us/step - loss: 0.1022
Epoch 472/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1022
Epoch 473/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1022
Epoch 474/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1022
Epoch 475/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1022
Epoch 476/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1022
Epoch 477/500
60000/60000 [==============================] - 0s 6us/step - loss: 0.1022
Epoch 478/500
60000/60000 [=====

In [117]:
print('還原圖')
reco=YDecoder.predict(XD_train)
show(reco[1:11])
print('Uniform生成')
a = np.random.random((10,10))
a = np.column_stack((np.identity(10),a))
reco = YDecoder.predict(a)
show(reco)
print('Normal生成')
a = np.random.normal(0.5,0.3,(10,10))
a = np.column_stack((np.identity(10),a))
reco = YDecoder.predict(a)
show(reco)

還原圖


Uniform生成


Normal生成
